In [1]:
import numpy as np 
import pandas as pd
import os
from keras.models import Sequential
from keras import layers
from keras.layers import BatchNormalization, Convolution2D , MaxPooling2D
from keras.layers.core import  Reshape , Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [1]:
df_train = pd.read_csv('../input/digit-recognizer/train.csv')
df_test = pd.read_csv('../input/digit-recognizer/test.csv')
target = df_train[["label"]]
feature = df_train.drop(columns=["label"])
x_train,x_test,y_train,y_test = train_test_split(feature,target,test_size=0.25)

In [1]:
y_train = pd.get_dummies(y_train.astype(str))
y_test = pd.get_dummies(y_test.astype(str))

In [1]:
net = Sequential()
net.add(layers.Dense(510,activation="relu",input_dim=784))
net.add(layers.Dense(100,activation='relu'))
net.add(layers.Dense(75,activation='relu'))
net.add(layers.Dense(60,activation='relu'))
net.add(layers.Dense(50,activation='selu'))
net.add(layers.Dense(25,activation='relu'))
net.add(layers.Dense(20,activation='selu'))
net.add(layers.Dense(15,activation='relu'))
net.add(layers.Dense(10,activation="softmax"))
net.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [1]:
model_net = net.fit(x_train,y_train,epochs=10,batch_size=64,validation_split=0.1)

In [1]:
score= net.evaluate(x_test,y_test,batch_size=64)
score

In [1]:
import matplotlib.pyplot as plt
pd.DataFrame(model_net.history).loc[:, ["loss", "val_loss"]].plot()
plt.xticks(range(10))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [1]:
net.summary()

# Convolutional

In [1]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
model = Sequential([
    Reshape((28, 28, 1)),
    Convolution2D(32,(3,3), activation='relu'),
    Convolution2D(32,(3,3), activation='relu'),
    MaxPooling2D(),
    Convolution2D(64,(3,3), activation='relu'),
    Convolution2D(64,(3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(200,activation='relu'),
    Dense(150,activation='relu'),
    Dense(100,activation='relu'),
    Dense(75,activation="selu"),
    Dense(50,activation='relu'),
    Dense(25,activation='relu'),
    Dense(10, activation='softmax')
])

In [1]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
con_net = model.fit(x=x_train, y=y_train, batch_size=64, validation_split=0.1, epochs=10)

In [1]:
pd.DataFrame(con_net.history).loc[:, ["loss", "val_loss"]].plot()
plt.xticks(range(10))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [1]:
y_test = y_test.astype(int)
model.evaluate(x=x_test, y=y_test, batch_size=64)

In [1]:
df_test = df_test.astype("float32")
predict = np.argmax(model.predict(df_test, verbose=0), axis=1)
submissions=pd.DataFrame({"ImageId": list(range(1,len(predict)+1)),
                         "Label": predict})
submissions.to_csv("submission.csv", index=False, header=True)